In [1]:
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import yaml


In [4]:
# plese labelling dataset of computer vision
# Define paths
dataset_path = "p/Users/kaleee/Documents/latihan-lks/pameran/bisindo/images"
output_path = "/Users/kaleee/Documents/latihan-lks/pameran/bisindo/labels"  # Directory to save YOLO format labels

# Create output directory if it doesn't exist
os.makedirs(output_path, exist_ok=True)

# Define sign language classes (maintain consistent order for class indices)
sign_classes = [
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J',
    'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T',
    'U', 'V', 'W', 'X', 'Y', 'Z',
    'space', 'delete', 'nothing'
]

# Save classes to yaml file for YOLO training
yaml_content = {
    'path': dataset_path,
    'train': 'train/images',
    'val': 'valid/images',
    'test': 'test/images',
    'nc': len(sign_classes),
    'names': sign_classes
}

with open('sign_dataset.yaml', 'w') as f:
    yaml.dump(yaml_content, f, sort_keys=False)

def create_yolo_label(image_path, bbox_coordinates):
    """
    Convert bounding box coordinates to YOLO format
    YOLO format: <class_index> <x_center> <y_center> <width> <height>
    All values are normalized between 0 and 1
    """
    img = cv2.imread(image_path)
    height, width = img.shape[:2]
    
    # bbox_coordinates should be in format: [x_min, y_min, x_max, y_max]
    x_min, y_min, x_max, y_max = bbox_coordinates
    
    # Convert to YOLO format
    x_center = ((x_min + x_max) / 2) / width
    y_center = ((y_min + y_max) / 2) / height
    bbox_width = (x_max - x_min) / width
    bbox_height = (y_max - y_min) / height
    
    return x_center, y_center, bbox_width, bbox_height

def label_sign_images():
    images = [f for f in os.listdir(dataset_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
    print(f"Found {len(images)} images to label")
    print("\nAvailable classes:", ', '.join(sign_classes))
    
    for img_name in images:
        img_path = os.path.join(dataset_path, img_name)
        img = cv2.imread(img_path)
        height, width = img.shape[:2]
        
        # Display image for labeling
        plt.figure(figsize=(10, 10))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title("Click and drag to draw bounding box")
        
        # Use matplotlib's rectangle selector for bounding box
        bbox = plt.ginput(2, timeout=-1)  # Get two corner points
        plt.close()
        
        if len(bbox) != 2:
            print("Skipping image due to incomplete bbox")
            continue
            
        # Convert bbox points to [x_min, y_min, x_max, y_max]
        x_min = min(bbox[0][0], bbox[1][0])
        x_max = max(bbox[0][0], bbox[1][0])
        y_min = min(bbox[0][1], bbox[1][1])
        y_max = max(bbox[0][1], bbox[1][1])
        
        # Get class label
        while True:
            class_name = input(f"Enter sign class for {img_name} (or 'q' to quit): ").upper()
            if class_name.lower() == 'q':
                return
            if class_name in sign_classes:
                break
            print("Invalid class! Please choose from the available classes.")
        
        class_idx = sign_classes.index(class_name)
        
        # Convert to YOLO format
        x_center, y_center, bbox_width, bbox_height = create_yolo_label(
            img_path, [x_min, y_min, x_max, y_max]
        )
        
        # Save label file (same name as image but .txt extension)
        label_path = os.path.join(output_path, os.path.splitext(img_name)[0] + '.txt')
        with open(label_path, 'w') as f:
            f.write(f"{class_idx} {x_center:.6f} {y_center:.6f} {bbox_width:.6f} {bbox_height:.6f}")
        
        print(f"Saved label for {img_name}")

# Create directory structure for YOLO
for split in ['train', 'valid', 'test']:
    for subdir in ['images', 'labels']:
        os.makedirs(os.path.join(dataset_path, split, subdir), exist_ok=True)

# Run the labeling function
# label_sign_images()

# After labeling, you'll need to:
# 1. Split your dataset into train/valid/test sets
# 2. Move images and their corresponding labels to appropriate directories
# 3. Use the generated sign_dataset.yaml file for YOLO training




In [12]:
# Split dataset into train/valid/test sets
import random
from shutil import copy2

# Get all image files
image_files = [f for f in os.listdir(output_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
random.shuffle(image_files)

# Split ratios
train_ratio = 0.7
valid_ratio = 0.2
test_ratio = 0.1

# Calculate split sizes
total = len(image_files)
train_size = int(total * train_ratio)
valid_size = int(total * valid_ratio)
test_size = total - train_size - valid_size

# Split the files
train_files = image_files[:train_size]
valid_files = image_files[train_size:train_size + valid_size]
test_files = image_files[train_size + valid_size:]

# Function to copy both image and its label
def copy_files(files, split):
    for f in files:
        # Copy image
        src_img = os.path.join(output_path, f)
        dst_img = os.path.join(dataset_path, split, 'images', f)
        copy2(src_img, dst_img)
        
        # Copy corresponding label
        label_file = os.path.splitext(f)[0] + '.txt'
        src_label = os.path.join(output_path, label_file)
        dst_label = os.path.join(dataset_path, split, 'labels', label_file)
        copy2(src_label, dst_label)

# Move files to respective directories
copy_files(train_files, 'train')
copy_files(valid_files, 'valid')
copy_files(test_files, 'test')

print(f"Dataset split complete:")
print(f"Train: {len(train_files)} images")
print(f"Valid: {len(valid_files)} images")
print(f"Test: {len(test_files)} images")

# Create YAML file for YOLO
yaml_content = f"""
path: {dataset_path}
train: train/images
val: valid/images
test: test/images

nc: {len(sign_classes)}
names: {sign_classes}
"""

with open(os.path.join(dataset_path, 'sign_dataset.yaml'), 'w') as f:
    f.write(yaml_content)

print("\nCreated sign_dataset.yaml file")


Dataset split complete:
Train: 0 images
Valid: 0 images
Test: 0 images

Created sign_dataset.yaml file
